# A Probabilistic Core of a Mobile App for Lottery Addiction

![gambling.png](online-gambling-word-cloud.png)

## Introduction:
Many people start playing the lottery for fun, but for some this activity turns into a habit which eventually escalates into addiction. Like other compulsive gamblers, lottery addicts soon begin spending from their savings and loans, they start to accumulate debts, and eventually engage in desperate behaviors like theft.



## Goal:
A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the [6/49 lottery](https://en.wikipedia.org/wiki/Lotto_6/49) and build functions that enable users to answer questions like:

- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

### Resources:
The institute also wants us to consider historical data coming from the national 6/49 lottery game in Canada. [The data set](https://www.kaggle.com/datascienceai/lottery-dataset) has data for 3,665 drawings, dating from 1982 to 2018 (we'll come back to this).

---

Throughout the project, we'll need to calculate repeatedly probabilities and combinations. As a consequence, we'll start by writing two functions that we'll use often:

- A function that calculates factorials; 

\begin{equation}
n! = n \times (n - 1) \times (n - 2) \times ... \times 2 \times 1
\end{equation}
- A function that calculates combinations.

\begin{equation}
_nC_k = {n \choose k} =  \frac{n!}{k!(n-k)!}
\end{equation}

In [25]:
def factorial(n):
    '''
    This function calculates factorial
    of the number
    
    Args:
        n : int
        
    Returns:
        int, factorial of the number
    '''
    final = 1
    for i in range(n, 0, -1): # start=n, stop=0, step=-1
        final *= i
    return final

In [35]:
def combinations(n,k):
    '''
    This function calculates number of
    combinations
    
    Args:
        n = total number of objects
        k = number of choosing objects
        
    Returns:
        int, number of combinations
    '''
    numerator = factorial(n)
    denominator = factorial(k) * factorial(n-k)
    formula = numerator/denominator
    return formula

---

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins the big prize if the six numbers on their tickets match all the six numbers drawn. If a player has a ticket with the numbers {13, 22, 24, 27, 42, 44}, he only wins the big prize if the numbers drawn are {13, 22, 24, 27, 42, 44}. If only one number differs, he doesn't win.

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

We discussed with the engineering team of the medical institute, and they told us we need to be aware of the following details when we write the function:

- Inside the app, the user inputs six different numbers from 1 to 49.
- Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function.
- The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.


Let's try to write this function!

In [87]:
def one_ticket_probability(list_of_6):
    '''
    This function calculates probability 
    of winning the lottery with single 
    ticket
    
    Args:
        list_of_6 : The six-digit Ticket number
    Returns:
        Nicely formatted chances to win
    '''
    total_outcomes = combinations(49,6)
    success_outcome = 1
    probability = success_outcome/total_outcomes
    prob_pc = probability * 100
    print('Hi! The chances that you will win with numbers {} are \n{:.7f}%\nThis means that out of {} chances, you have a single chance to win'.format(list_of_6, prob_pc, int(1/probability)))
    print('So how Lucky are you feeling?')


In [88]:
# Lets try out our function
one_ticket_probability([2,3,4,5,6,7])

Hi! The chances that you will win with numbers [2, 3, 4, 5, 6, 7] are 
0.0000072%
This means that out of 13983816 chances, you have a single chance to win
So how Lucky are you feeling?


---

For the first version of the app, however, users should also be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

So, lets explore the historical data coming from the **Canada 6/49 lottery**.

## Import Libraries and Load Dataset

In [93]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


In [94]:
# Load Dataset
data = pd.read_csv('649.csv')

In [95]:
data.head()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34
3,649,4,0,7/3/1982,3,9,10,13,20,43,34
4,649,5,0,7/10/1982,5,14,21,31,34,47,45


In [97]:
data.tail()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3660,649,3587,0,6/6/2018,10,15,23,38,40,41,35
3661,649,3588,0,6/9/2018,19,25,31,36,46,47,26
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


In [99]:
data.shape

(3665, 11)

In [98]:
data.columns

Index(['PRODUCT', 'DRAW NUMBER', 'SEQUENCE NUMBER', 'DRAW DATE',
       'NUMBER DRAWN 1', 'NUMBER DRAWN 2', 'NUMBER DRAWN 3', 'NUMBER DRAWN 4',
       'NUMBER DRAWN 5', 'NUMBER DRAWN 6', 'BONUS NUMBER'],
      dtype='object')

The data set contains historical data for 3,665 drawings (each row shows data for a single drawing), dating from 1982 to 2018. For each drawing, we can find the six numbers drawn in the following six columns:

1. `NUMBER DRAWN 1`
2. `NUMBER DRAWN 2`
3. `NUMBER DRAWN 3`
4. `NUMBER DRAWN 4`
5. `NUMBER DRAWN 5`
6. `NUMBER DRAWN 6`

---

### What are the chances that a ticket number has won in past?
We're going to write a function that will enable users to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

The engineering team told us that we need to be aware of the following details:

- Inside the app, the user inputs six different numbers from 1 to 49.
- Under the hood, the six numbers will come as a Python list and serve as an input to our function.
- The engineering team wants us to write a function that prints:
    - the number of times the combination selected occurred in the Canada data set; and
    - the probability of winning the big prize in the next drawing with that combination.

We'll now start working on writing this function. 


In [116]:
def extract_numbers(row):
    '''
    A simple function that extracts data from 
    the given rows
    '''
    row = row[4:10]  # index 5-10 contains the numbers
    row = set(row) # remove repeatations
    return row
winning_numbers = data.apply(extract_numbers, axis=1)
winning_numbers.head(5)

In [139]:
def check_historical_occurences(user_numbers, winning_numbers):
    '''
    The functions checks if a given combination 
    of numbers ever occured in past
    
    Args:
        user_numbers: The numbers to be checked
        winning_numbers: All winning combinations 
                         from the past.
    Returns:
        If the combination ever occured. If it did
        then how many times
    '''
    user_numbers = set(user_numbers)
    have_occured = user_numbers == winning_numbers
    freq_occured = have_occured.sum()
    
    if freq_occured > 0:
        print('This combination of 6 numbers has occurred {} time(s) in the past.'.format(freq_occured))
        print('But your winning odds are the same for the next time,\nas you have a 0.0000072% chance of winning the big prize in the next \ndrawing when you use this combination of numbers.')
        
    else:
        print('This combination of 6 numbers has never occurred in the past.')
        print('But it doesn\'t mean it is likely to occur now. You have a \n0.0000072% chance of winning the big prize in the next drawing\nwhen you use this combination of numbers.')

In [140]:
# Testing
testset_1 = [33, 36, 37, 39, 8, 41]
check_historical_occurences(testset_1, winning_numbers)  # knowingly using an already occured combination

This combination of 6 numbers has occurred 1 time(s) in the past.
But your winning odds are the same for the next time,
as you have a 0.0000072% chance of winning the big prize in the next 
drawing when you use this combination of numbers.


In [142]:
# Testing
testset_2 = [1, 2, 3, 4, 5, 5]
check_historical_occurences(testset_2, winning_numbers)

This combination of 6 numbers has never occurred in the past.
But it doesn't mean it is likely to occur now. You have a 
0.0000072% chance of winning the big prize in the next drawing
when you use this combination of numbers.


---

### What if a person has multiple Tickets?
Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning .
We've talked with the engineering team and they gave us the following information:

- The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
- Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
- The function should print information about the probability of winning the big prize depending on the number of different tickets played.

In [170]:
def multi_ticket_probability(num_tickets):
     '''
    This function calculates  probability of 
    winning with a multiple number of tickets.
    
    Args:
        num_tickets: Number of tickets
    Returns:
        Nicely formatted chances to win
    '''
    total_outcomes = combinations(49,6)
    success_outcomes = num_tickets # same as number of tickets.
    
    probability = success_outcomes/total_outcomes
    prob_pc = probability * 100
    
    if num_tickets == 1:
        print('Hi! Your chances to win big prize with {} number of different tickets are {:.6f}%\nIn other words, you have 1 chance out of {} chances to win big prize'.format(num_tickets, prob_pc, round(1/probability)))
    
    else:
        print('Hi! Your chances to win big prize with {} number of different tickets are {:.6f}%\nIn other words, you have {} chance out of {} chances to win big prize'.format(num_tickets, prob_pc, num_tickets, int(num_tickets*(1/probability))))


In [184]:
# Testing

# Following is the list with numbers of tickets a person could buy. Note that its not a ticket number!
number_of_tickets = [1, 10, 100, 10000, 1000000, 6991908, 13983816]

for quantity in number_of_tickets:
    print('I bought {} tickets. What are my chances to win?'.format(quantity))
    print('\n')
    multi_ticket_probability(quantity)
    print('\n')
    print('-'*100)
    

I bought 1 tickets. What are my chances to win?


Hi! Your chances to win big prize with 1 number of different tickets are 0.000007%
In other words, you have 1 chance out of 13983816 chances to win big prize


----------------------------------------------------------------------------------------------------
I bought 10 tickets. What are my chances to win?


Hi! Your chances to win big prize with 10 number of different tickets are 0.000072%
In other words, you have 10 chance out of 13983816 chances to win big prize


----------------------------------------------------------------------------------------------------
I bought 100 tickets. What are my chances to win?


Hi! Your chances to win big prize with 100 number of different tickets are 0.000715%
In other words, you have 100 chance out of 13983816 chances to win big prize


----------------------------------------------------------------------------------------------------
I bought 10000 tickets. What are my chances to win?


Hi! 

---

### What if some of the 6 numbers on a ticket match the winning numbers?
In most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

These are the engineering details we'll need to be aware of:

- Inside the app, the user inputs:
    - six different numbers from 1 to 49; and
    - an integer between 2 and 5 that represents the number of winning numbers expected

In [188]:
def probability_less_6(num_matched):
    '''
    This function calculates the probability
    of the exact number of digits to match with
    the 6 winning digits on a ticket.
    
    Args:
        num_matched: Number of the digits on a 
                     ticket to match the numbers 
                     of digits of winning ticket.
                     
    Returns:
        Nicely formatted chances of matching exact
        'num_matched' numbers of digits with the
        winning digits combination
    '''
    
    if num_matched < 2 or num_matched >5: # Reject if invalid input
        return 'You should input a number between 2 and 5 inclusive'
    
    else:
        combinations_matched = combinations(6, num_matched)
        combinations_left = combinations(43, 6 - num_matched)
        successful_outcomes = combinations_matched * combinations_left

        total_outcomes = combinations(49, 6)    
        probability = successful_outcomes / total_outcomes

        prob_pc = probability * 100    
        combinations_success = round(total_outcomes/successful_outcomes)    
        print('Hi!, your chances of having exactly {} winning numbers on this ticket are {:.6f}%. In other words, you have a 1 in {:,} chances to win.'.format(num_matched, prob_pc, int(combinations_success)))

In [196]:
# Testing 
number_of_digits = [2, 3, 4, 5]
for to_match in number_of_digits:
    print('What are my chances to have exactly {} digits of my ticket to be matched with the winning combination?'.format(to_match))
    print('\n')
    probability_less_6(to_match)
    print('\n')
    print('-'*125)


What are my chances to have 2 digits of my ticket to be matched with the winning combination?


Hi!, your chances of having 2 winning numbers on this ticket are 13.237803%. In other words, you have a 1 in 8 chances to win.


-----------------------------------------------------------------------------------------------------------------------------
What are my chances to have 3 digits of my ticket to be matched with the winning combination?


Hi!, your chances of having 3 winning numbers on this ticket are 1.765040%. In other words, you have a 1 in 57 chances to win.


-----------------------------------------------------------------------------------------------------------------------------
What are my chances to have 4 digits of my ticket to be matched with the winning combination?


Hi!, your chances of having 4 winning numbers on this ticket are 0.096862%. In other words, you have a 1 in 1,032 chances to win.


----------------------------------------------------------------------

In [215]:
def probability_atleast(atleast_num):
    '''
    This function calculates the probability
    of matching atleast the given number of 
    digits to match the winning combination.
    
    Args:
        num_matched: Number of the digits on a 
                     ticket to match the numbers 
                     of digits of winning ticket.
                     
    Returns:
        Nicely formatted chances of matching 
        'num_matched' numbers of digits with the
        winning digits combination
    '''
    num_success_outcomes = 0
    
    for i in range(atleast_num, 6+1):
        combinations_matched = combinations(6, i)
        combinations_left = combinations(43, 6-i)
        success_outcomes = combinations_matched * combinations_left
        num_success_outcomes += success_outcomes
        
    total_outcomes = combinations(49, 6)
    
    probability = num_success_outcomes / total_outcomes
    round_prob_pc = round((num_success_outcomes/total_outcomes * 100)) 
    
    print('Hi!, your chances of having atleast {} winning numbers on this ticket are {:.6f}%. In other words, you have a 1 in {} chances to win.'.format(atleast_num, probability, int(total_outcomes/num_success_outcomes)))

In [216]:
for winning_num in [2,3,4,5]:
    probability_atleast(winning_num)
    print("============================")

Hi!, your chances of having atleast 2 winning numbers on this ticket are 0.151016%. In other words, you have a 1 in 6 chances to win.
Hi!, your chances of having atleast 3 winning numbers on this ticket are 0.018638%. In other words, you have a 1 in 53 chances to win.
Hi!, your chances of having atleast 4 winning numbers on this ticket are 0.000987%. In other words, you have a 1 in 1013 chances to win.
Hi!, your chances of having atleast 5 winning numbers on this ticket are 0.000019%. In other words, you have a 1 in 53991 chances to win.


In [218]:
# Testing 
number_of_digits = [2, 3, 4, 5]
for atleast_match in number_of_digits:
    print('What are my chances to have exactly {} digits of my ticket to be matched with the winning combination?'.format(atleast_match))
    print('\n')
    probability_atleast(atleast_match)
    print('\n')
    print('-'*125)

What are my chances to have exactly 2 digits of my ticket to be matched with the winning combination?


Hi!, your chances of having atleast 2 winning numbers on this ticket are 0.151016%. In other words, you have a 1 in 6 chances to win.


-----------------------------------------------------------------------------------------------------------------------------
What are my chances to have exactly 3 digits of my ticket to be matched with the winning combination?


Hi!, your chances of having atleast 3 winning numbers on this ticket are 0.018638%. In other words, you have a 1 in 53 chances to win.


-----------------------------------------------------------------------------------------------------------------------------
What are my chances to have exactly 4 digits of my ticket to be matched with the winning combination?


Hi!, your chances of having atleast 4 winning numbers on this ticket are 0.000987%. In other words, you have a 1 in 1013 chances to win.


------------------------